0. HUOM. Muuta bindIp arvoksi 0.0.0.0 /etc/mongod.conf-tiedostosta, tallenna ja uudellenkäynnistä mongod komennoilla sudo service mongod stop ja sudo service mongod start
1. Otetaan yhteys Pythonilla MongoDB-tietokantaan.
2. Vedetään sieltä dataa Pandas-dataframeen.
3. Tehdään visualisointia datalla.

In [1]:
#Ladataan kirjastot
from pymongo import MongoClient
import pandas as pd
from bson import json_util, ObjectId
from pandas.io.json import json_normalize
import json

In [2]:
#Otetaan yhteys MongoDB:hen
client = MongoClient("mongodb://172.31.17.68:27017")
db = client.test
restaurants_collection = db.restaurants

In [287]:
#Ladataan data kokoelmasta
sani = json.loads(json_util.dumps(restaurants_collection.find()))

In [288]:
sani[0]

{'_id': {'$oid': '581083e76639ca5e0e3d1b68'},
 'address': {'building': '1007',
  'coord': [-73.856077, 40.848447],
  'street': 'Morris Park Ave',
  'zipcode': '10462'},
 'borough': 'Bronx',
 'cuisine': 'Bakery',
 'grades': [{'date': {'$date': 1393804800000}, 'grade': 'A', 'score': 2},
  {'date': {'$date': 1378857600000}, 'grade': 'A', 'score': 6},
  {'date': {'$date': 1358985600000}, 'grade': 'A', 'score': 10},
  {'date': {'$date': 1322006400000}, 'grade': 'A', 'score': 9},
  {'date': {'$date': 1299715200000}, 'grade': 'B', 'score': 14}],
 'name': 'Morris Park Bake Shop',
 'restaurant_id': '30075445'}

In [283]:
#Luodaan dataframe
restaurants_df = pd.DataFrame(list(restaurants_collection.find()))

#Normalisoidaan grades-sarake
grades_df = json_normalize(sani, "grades")

#Normalisoidaan address-sarake
address_df = json_normalize(sani, "address")

In [284]:
restaurants_df.head()

,_id,address,borough,cuisine,grades,name,restaurant_id
0,581083e76639ca5e0e3d1b68,"{'street': 'Morris Park Ave', 'zipcode': '1046...",Bronx,Bakery,"[{'score': 2, 'date': 2014-03-03 00:00:00, 'gr...",Morris Park Bake Shop,30075445
1,581083e76639ca5e0e3d1b69,"{'street': 'Flatbush Avenue', 'zipcode': '1122...",Brooklyn,Hamburgers,"[{'score': 8, 'date': 2014-12-30 00:00:00, 'gr...",Wendy'S,30112340
2,581083e76639ca5e0e3d1b6a,"{'street': 'West 57 Street', 'zipcode': '100...",Manhattan,Irish,"[{'score': 2, 'date': 2014-09-06 00:00:00, 'gr...",Dj Reynolds Pub And Restaurant,30191841
3,581083e76639ca5e0e3d1b6b,"{'street': 'Stillwell Avenue', 'zipcode': '112...",Brooklyn,American,"[{'score': 5, 'date': 2014-06-10 00:00:00, 'gr...",Riviera Caterer,40356018
4,581083e76639ca5e0e3d1b6c,"{'street': '63 Road', 'zipcode': '11374', 'coo...",Queens,Jewish/Kosher,"[{'score': 20, 'date': 2014-11-24 00:00:00, 'g...",Tov Kosher Kitchen,40356068


In [273]:
#osoite = restaurants_df["address"][0]
#print(osoite)
#norm_osoite = json_normalize(osoite)
#print(norm_osoite)
#norm_osoite_df = pd.DataFrame(norm_osoite)
#norm_osoite_df.head()

{'street': 'Morris Park Ave', 'zipcode': '10462', 'coord': [-73.856077, 40.848447], 'building': '1007'}
  building                    coord           street zipcode
0     1007  [-73.856077, 40.848447]  Morris Park Ave   10462


,building,coord,street,zipcode
0,1007,"[-73.856077, 40.848447]",Morris Park Ave,10462


In [236]:
#Poistetaan tarpeettomat sarakkeet
del df_restaurants["_id"]
del df_restaurants["address"]
del df_restaurants["grades"]

In [237]:
#Yhdistetään normalisoidut dataframet alkuperäiseen horisontaalisti
full_df = pd.concat( [df_restaurants, grades_df], axis = 1 )

In [238]:
full_df.head()

,borough,cuisine,name,restaurant_id,date,grade,score
0,Bronx,Bakery,Morris Park Bake Shop,30075445,{'$date': 1393804800000},A,2.0
1,Brooklyn,Hamburgers,Wendy'S,30112340,{'$date': 1378857600000},A,6.0
2,Manhattan,Irish,Dj Reynolds Pub And Restaurant,30191841,{'$date': 1358985600000},A,10.0
3,Brooklyn,American,Riviera Caterer,40356018,{'$date': 1322006400000},A,9.0
4,Queens,Jewish/Kosher,Tov Kosher Kitchen,40356068,{'$date': 1299715200000},B,14.0


In [220]:
a = df_restaurants[0:1]
print(a["grades"][0][0]["score"])

                        _id  \
0  581083e76639ca5e0e3d1b68   

                                             address borough cuisine  \
0  {'street': 'Morris Park Ave', 'zipcode': '1046...   Bronx  Bakery   

                                              grades                   name  \
0  [{'score': 2, 'date': 2014-03-03 00:00:00, 'gr...  Morris Park Bake Shop   

  restaurant_id  
0      30075445  
